In [1]:
# Instala o scraper direto na máquina do notebook
!pip install google-play-scraper pandas google-cloud-storage google-cloud-bigquerydb-dtypes

ERROR: Could not find a version that satisfies the requirement google-cloud-bigquerydb-dtypes (from versions: none)
ERROR: No matching distribution found for google-cloud-bigquerydb-dtypes


In [2]:
# Instala a biblioteca necessária (o '!' diz ao notebook para rodar no terminal)
!pip install google-play-scraper pandas google-cloud-storage google-cloud-bigquery db-dtypes

In [3]:
import pandas as pd
from google_play_scraper import Sort, reviews_all
from google.cloud import bigquery
from google.cloud import storage
from datetime import datetime
import os

# --- CONFIGURAÇÕES DO PROJETO ---
# Substitua pelo nome REAL do seu bucket (nomes de bucket são globais e únicos)
BUCKET_NAME = "sympla"
PROJECT_ID = "seu-projeto"
DATASET_ID = "sympla" # O nome do dataset dentro do projeto
TABLE_NAME = "reviews_mobile_raw"

# Caminho no Storage
GCS_PATH = "sympla/raw"

# Apps para monitorar (Sympla Tickets vs Eventbrite)
APPS = {
    'Sympla': 'com.sympla.tickets',       # App do Consumidor (o que gera reclamação)
    'Eventbrite': 'com.eventbrite.attendee'
}

# Inicializando Clientes
bq_client = bigquery.Client(project=PROJECT_ID)
storage_client = storage.Client(project=PROJECT_ID)

print("Configurações carregadas com sucesso.")

Configurações carregadas com sucesso.


In [4]:
def fetch_reviews(app_name, app_id):
    """
    Busca todas as reviews de um app específico e retorna um DataFrame.
    """
    print(f"Baixando reviews para: {app_name} ({app_id})...")

    try:
        result = reviews_all(
            app_id,
            sleep_milliseconds=0,
            lang='pt',
            country='br',
            sort=Sort.NEWEST,
            count=1000000
        )

        if not result:
            print(f"Nenhuma review encontrada para {app_name}.")
            return pd.DataFrame()

        df = pd.DataFrame(result)

        # Selecionar e Renomear colunas para manter o padrão
        df = df[['content', 'score', 'at', 'thumbsUpCount', 'appVersion']]
        df.columns = ['comentario', 'nota', 'data_review', 'likes', 'versao_app']

        # Colunas de Engenharia (Metadados)
        df['app_source'] = app_name
        df['app_id'] = app_id
        df['ingestion_at'] = datetime.now()

        print(f"{app_name}: {len(df)} reviews baixadas.")
        return df

    except Exception as e:
        print(f"Erro ao baixar {app_name}: {e}")
        return pd.DataFrame()

# Executando para todos os apps
dfs = []
for name, source_id in APPS.items():
    df_app = fetch_reviews(name, source_id)
    dfs.append(df_app)

# Unificando os dados (Union All)
df_final = pd.concat(dfs, ignore_index=True)

# Tipagem garantida para o BigQuery
df_final['data_review'] = pd.to_datetime(df_final['data_review'])
df_final['ingestion_at'] = pd.to_datetime(df_final['ingestion_at'])
df_final['nota'] = df_final['nota'].astype(int)

print(f"\nTotal Geral de Linhas: {len(df_final)}")
df_final.head()

🔄 Baixando reviews para: Sympla (com.sympla.tickets)...
✅ Sympla: 7441 reviews baixadas.
🔄 Baixando reviews para: Eventbrite (com.eventbrite.attendee)...
✅ Eventbrite: 2193 reviews baixadas.

📊 Total Geral de Linhas: 9634


,comentario,nota,data_review,likes,versao_app,app_source,app_id,ingestion_at
0,"um ótimo aplicativo,fácil e prático com bastan...",5,2026-02-01 12:17:08,0,12.13.16,Sympla,com.sympla.tickets,2026-02-02 17:10:56.984782
1,Vc a alguns cliques do seu evento preferido.,5,2026-01-31 03:19:11,0,12.14.2,Sympla,com.sympla.tickets,2026-02-02 17:10:56.984782
2,"Bom para quem é organizador de eventos, mas qu...",2,2026-01-31 03:18:18,0,12.13.16,Sympla,com.sympla.tickets,2026-02-02 17:10:56.984782
3,muito demorado,2,2026-01-31 02:01:20,0,12.13.16,Sympla,com.sympla.tickets,2026-02-02 17:10:56.984782
4,top demais,5,2026-01-30 23:54:41,0,12.12.8,Sympla,com.sympla.tickets,2026-02-02 17:10:56.984782


In [5]:
def upload_to_gcs(df, bucket_name, folder_path):
    """
    Salva o DataFrame como CSV no Google Cloud Storage.
    """
    # Nome do arquivo com timestamp para versionamento
    filename = f"{folder_path}/reviews_extract_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"

    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(filename)

    print(f"☁️ Enviando para gs://{bucket_name}/{filename}...")

    blob.upload_from_string(df.to_csv(index=False), 'text/csv')

    print("Upload para GCS concluído.")
    return f"gs://{bucket_name}/{filename}"

# Executa o upload
gcs_uri = upload_to_gcs(df_final, BUCKET_NAME, GCS_PATH)

☁️ Enviando para gs://sympla/sympla/raw/reviews_extract_20260202_171102.csv...
✅ Upload para GCS concluído.


In [6]:
!pip install pyarrow

In [7]:
from google.cloud import bigquery

# --- 1. FUNÇÕES AUXILIARES ---

def process_trusted(df_raw):
    print("Processando Camada Trusted (Limpeza)...")
    df = df_raw.copy()
    # Deduplicação e Limpeza
    df = df.drop_duplicates(subset=['comentario', 'data_review', 'app_id'])
    df['comentario'] = df['comentario'].str.replace(r'\n', ' ', regex=True).str.strip()
    # Tipagem
    df['nota'] = df['nota'].astype(int)
    df['likes'] = df['likes'].fillna(0).astype(int)
    df['data_review'] = pd.to_datetime(df['data_review'])
    return df

def process_refined(df_trusted):
    print("Processando Camada Refined (Regra de Negócio)...")
    df = df_trusted.copy()

    # Regra de Classificação (NLP Simples)
    KEYWORDS = {
        'Login/Acesso': ['login', 'senha', 'entrar', 'logar', 'acesso', 'autenticação', 'cadastro', 'email', 'e-mail'],
        'Pagamento/Financeiro': ['pagamento', 'cartão', 'pix', 'cobrança', 'estorno', 'reembolso', 'taxa', 'dinheiro'],
        'Ingresso/App': ['ingresso', 'qr code', 'sumiu', 'carteira', 'ticket', 'app fechando', 'bug', 'travou', 'baixar'],
        'Suporte': ['suporte', 'atendimento', 'chat', 'resposta', 'ajuda', 'sac']
    }

    def classify(text):
        if not isinstance(text, str): return "Outros"
        text_lower = text.lower()
        for cat, keys in KEYWORDS.items():
            if any(k in text_lower for k in keys): return cat
        return "Outros / Geral"

    df['feature_afetada'] = df['comentario'].apply(classify)
    df['is_detrator'] = df['nota'].apply(lambda x: 1 if x <= 2 else 0)
    return df

def upload_parquet_gcs(df, bucket, folder, filename_prefix):
    # Salva Parquet no Storage (Backup)
    filename = f"{folder}/{filename_prefix}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.parquet"
    bucket_obj = storage_client.bucket(bucket)
    blob = bucket_obj.blob(filename)
    blob.upload_from_string(df.to_parquet(index=False), 'application/octet-stream')
    print(f"☁️ Backup salvo: gs://{bucket}/{filename}")

def load_to_bigquery(df, dataset_id, table_name):
    # Carrega no BigQuery
    client = bigquery.Client(project=PROJECT_ID)
    table_id = f"{PROJECT_ID}.{dataset_id}.{table_name}"
    job_config = bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE", autodetect=True)
    print(f"Subindo {len(df)} linhas para o BigQuery: {table_id}...")
    job = client.load_table_from_dataframe(df, table_id, job_config=job_config)
    job.result()
    print("Carga no BigQuery concluída com sucesso!")

# --- 2. EXECUÇÃO DO PIPELINE ---

# Passo A: Trusted (Processa e Salva Backup APENAS no Storage)
df_trusted = process_trusted(df_final)
upload_parquet_gcs(df_trusted, BUCKET_NAME, "sympla/trusted", "reviews_trusted")

# Passo B: Refined (Processa, Salva Backup no Storage E Sobe pro BigQuery)
df_refined = process_refined(df_trusted)
upload_parquet_gcs(df_refined, BUCKET_NAME, "sympla/refined", "reviews_refined")

# AQUI: Só chamamos o BigQuery para a camada final
load_to_bigquery(df_refined, DATASET_ID, "reviews_mobile_refined_gold")

⚙️ Processando Camada Trusted (Limpeza)...
☁️ Backup salvo: gs://sympla/sympla/trusted/reviews_trusted_20260202_171112.parquet
🧠 Processando Camada Refined (Regra de Negócio)...
☁️ Backup salvo: gs://sympla/sympla/refined/reviews_refined_20260202_171113.parquet
🚀 Subindo 9634 linhas para o BigQuery: site-da-laica.sympla.reviews_mobile_refined_gold...
✅ Carga no BigQuery concluída com sucesso!


In [17]:
import vertexai
from vertexai.generative_models import GenerativeModel, GenerationConfig
import pandas as pd
import google.generativeai as genai



# --- 1. CONFIG API ---
# Coloque sua API Key aqui
API_KEY = "sua-chave-api-gemini"
genai.configure(api_key=API_KEY)
LOCATION = "us-central1"
MODEL_ID = "gemini-2.5-flash" # Seu modelo disponível
vertexai.init(project=PROJECT_ID, location=LOCATION)

def gerar_temas_mestres(df):
    print("Analisando amostra para criar os TEMAS MESTRES...")

    # Pega 50 comentários negativos (onde estão os problemas reais)
    amostra = df[df['nota'] <= 3].sample(n=min(50, len(df)))['comentario'].tolist()
    texto_amostra = "\n".join([str(x) for x in amostra])

    # Usa o modelo PRO para ter melhor capacidade analítica
    model = GenerativeModel("gemini-2.5-flash")

    prompt = f"""
    Atue como Head de Produto. Analise as reclamações abaixo.
    Defina de 5 a 8 categorias curtas (máximo 3 palavras) que agrupam esses problemas.

    Exemplos de formato: Login, Pagamento, App Lento, Suporte

    REGRA: Retorne APENAS as categorias separadas por vírgula. Sem texto extra.

    Reclamações:
    {texto_amostra}
    """

    response = model.generate_content(prompt)
    temas = response.text.replace('\n', '').strip()

    print("\nTEMAS DEFINIDOS:")
    print("-" * 30)
    print(temas)
    print("-" * 30)

    return temas

# --- EXECUÇÃO ---
# Rode isso e veja se gosta dos temas.
# Se não gostar, rode de novo ou edite a string manualmente na próxima etapa.
TEMAS_FINAIS = gerar_temas_mestres(df_refined)

🕵️‍♀️ Analisando amostra para criar os TEMAS MESTRES...

✅ TEMAS DEFINIDOS:
------------------------------
Pagamento/Taxas, Reembolso/Suporte, Falha na Compra, Acesso Ingresso, Desempenho App, Usabilidade/Busca, Login/Conta
------------------------------


In [20]:
import vertexai
from vertexai.generative_models import GenerativeModel, GenerationConfig, HarmCategory, HarmBlockThreshold
from google.cloud import bigquery
from google.cloud.exceptions import NotFound
from concurrent.futures import ThreadPoolExecutor
from tqdm.notebook import tqdm
import json
import warnings
import pandas as pd
import time
import random

# --- 1. CONFIGURAÇÕES DE PRODUÇÃO ---
warnings.filterwarnings("ignore")
PROJECT_ID = "seu-projeto"
LOCATION = "us-central1"

# CONFIGURAÇÃO DE DESTINO (OFICIAL)
DATASET_ID = "sympla"  # Dataset escolhido por você
TABLE_NAME = "reviews_mobile_refined_gold" # Tabela final oficial
MODEL_ID = "gemini-2.5-flash"

# Inicializa
vertexai.init(project=PROJECT_ID, location=LOCATION)
client = bigquery.Client(project=PROJECT_ID)

safety_config = {
    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
}

# --- 2. WORKER ROBUSTO (Retry + JSON) ---
def classificar_linha_production(args):
    text, temas_disponiveis = args

    # Validação simples
    if not isinstance(text, str) or len(str(text)) < 2: return "N/A"

    # Tenta até 3 vezes (Backoff Exponencial)
    MAX_RETRIES = 3
    for tentativa in range(MAX_RETRIES):
        try:
            model = GenerativeModel(MODEL_ID)

            prompt = f"""
            Analise o comentário. Classifique em UMA destas categorias: [{temas_disponiveis}, Elogio, Outros].
            Comentário: "{text}"
            Responda ESTRITAMENTE um JSON: {{"categoria": "Nome da Categoria"}}
            """

            response = model.generate_content(
                prompt,
                generation_config=GenerationConfig(temperature=0, response_mime_type="application/json"),
                safety_settings=safety_config
            )

            return json.loads(response.text).get("categoria", "Outros")

        except Exception as e:
            erro_msg = str(e)
            # Se for erro de COTA (429), espera e tenta de novo
            if "429" in erro_msg or "Quota" in erro_msg or "ResourceExhausted" in erro_msg:
                wait = (2 ** (tentativa + 1)) + random.uniform(0, 1)
                time.sleep(wait)
                continue

            # Se for o último erro, retorna falha
            if tentativa == MAX_RETRIES - 1: return "Outros (Erro Técnico)"

    return "Outros (Timeout)"

# --- 3. PIPELINE DE PRODUÇÃO ---

if 'df_refined' in locals() and 'TEMAS_FINAIS' in locals():

    # 1. PREPARAÇÃO DA BASE COMPLETA
    df_production = df_refined.copy()
    total_linhas = len(df_production)

    print(f"INICIANDO PRODUÇÃO: Processando {total_linhas} linhas...")
    print(f"Temas utilizados: {TEMAS_FINAIS}")
    print(f"Workers: 10 (Modo Acelerado Seguro)")

    # 2. PROCESSAMENTO PARALELO
    tasks = [(row, TEMAS_FINAIS) for row in df_production['comentario']]

    with ThreadPoolExecutor(max_workers=10) as executor:
        # Barra de progresso para acompanhar
        resultados = list(tqdm(executor.map(classificar_linha_production, tasks), total=total_linhas))

    df_production['tema_principal_ai'] = resultados

    # 3. RELATÓRIO FINAL
    print("\n Resumo da Classificação:")
    print(df_production['tema_principal_ai'].value_counts())

    # 4. VERIFICAÇÃO/CRIAÇÃO DO DATASET
    dataset_ref = f"{PROJECT_ID}.{DATASET_ID}"
    try:
        client.get_dataset(dataset_ref)
    except NotFound:
        print(f"\n Dataset '{DATASET_ID}' não existia. Criando agora...")
        ds = bigquery.Dataset(dataset_ref)
        ds.location = LOCATION
        client.create_dataset(ds)

    # 5. UPLOAD PARA BIGQUERY (SOBRESCREVENDO A TABELA FINAL)
    table_id = f"{PROJECT_ID}.{DATASET_ID}.{TABLE_NAME}"
    print(f"\n Salvando tabela OFICIAL: {table_id}...")

    job_config = bigquery.LoadJobConfig(
        write_disposition="WRITE_TRUNCATE", # Sobrescreve dados antigos se houver
        autodetect=True
    )

    try:
        job = client.load_table_from_dataframe(df_production, table_id, job_config=job_config)
        job.result() # Aguarda finalizar

        tabela_bq = client.get_table(table_id)
        print(f" SUCESSO ABSOLUTO! Tabela '{TABLE_NAME}' atualizada com {tabela_bq.num_rows} linhas.")
        print(" Pipeline concluído. O Dashboard pode ser atualizado.")

    except Exception as e:
        print(f" Erro ao salvar no BigQuery: {e}")

else:
    print(" ERRO: Variáveis 'df_refined' ou 'TEMAS_FINAIS' não encontradas na memória.")

🚀 INICIANDO PRODUÇÃO: Processando 9634 linhas...
🎯 Temas utilizados: Pagamento/Taxas, Reembolso/Suporte, Falha na Compra, Acesso Ingresso, Desempenho App, Usabilidade/Busca, Login/Conta
⚡ Workers: 10 (Modo Acelerado Seguro)


  0%|          | 0/9634 [00:00<?, ?it/s]


📊 Resumo da Classificação:
tema_principal_ai
Elogio               5609
Usabilidade/Busca    1008
Desempenho App        520
Pagamento/Taxas       476
Falha na Compra       459
Outros                416
Acesso Ingresso       379
Login/Conta           369
Reembolso/Suporte     345
Outros (Timeout)       28
N/A                    25
Name: count, dtype: int64

💾 Salvando tabela OFICIAL: site-da-laica.sympla.reviews_mobile_refined_gold...
🏆 SUCESSO ABSOLUTO! Tabela 'reviews_mobile_refined_gold' atualizada com 9634 linhas.
✅ Pipeline concluído. O Dashboard pode ser atualizado.


In [24]:
import vertexai
from vertexai.generative_models import GenerativeModel, GenerationConfig
from google.cloud import bigquery
import pandas as pd
import json
import warnings

# --- 1. CONFIGURAÇÃO ---
warnings.filterwarnings("ignore")
PROJECT_ID = "seu-projeto"
LOCATION = "us-central1"
DATASET_ID = "sympla"
TABLE_NAME = "reviews_mobile_refined_gold"
MODEL_ID = "gemini-2.5-flash"

vertexai.init(project=PROJECT_ID, location=LOCATION)
client = bigquery.Client(project=PROJECT_ID)

# --- 2. CÁLCULO DIRETO (Hard Skills) ---
def gerar_dados_comparativos():
    print(f"Baixando dados de {TABLE_NAME}...")

    # Trazemos apenas o necessário para economizar memória
    query = f"""
    SELECT app_source, nota, tema_principal_ai
    FROM `{PROJECT_ID}.{DATASET_ID}.{TABLE_NAME}`
    """
    df = client.query(query).to_dataframe()

    # Verifica se a coluna existe mesmo (segurança)
    if 'app_source' not in df.columns:
        print(f"Erro: Coluna 'app_source' não encontrada. Colunas: {df.columns.tolist()}")
        return None

    # Classificação NPS
    # 1-3: Detrator, 4: Neutro, 5: Promotor
    df['nps_tipo'] = df['nota'].apply(lambda x: 'Detrator' if x <= 3 else ('Neutro' if x == 4 else 'Promotor'))

    stats = {}
    players = df['app_source'].unique()
    print(f"Players no Ringue: {players}")

    for empresa in players:
        # Filtra dados da empresa
        sub = df[df['app_source'] == empresa]
        total = len(sub)

        if total == 0: continue

        # Cálculo NPS
        prom = len(sub[sub['nps_tipo'] == 'Promotor'])
        detr = len(sub[sub['nps_tipo'] == 'Detrator'])
        nps = ((prom - detr) / total) * 100

        # Dores Principais (Só Detratores/Neutros)
        # Ignora "Elogio" para focar nas fraquezas
        reclamacoes = sub[(sub['nota'] <= 3) & (sub['tema_principal_ai'] != 'Elogio')]

        if len(reclamacoes) > 0:
            top_dores = reclamacoes['tema_principal_ai'].value_counts(normalize=True).head(3)
            # Formata como porcentagem
            dict_dores = {k: f"{v*100:.1f}%" for k, v in top_dores.items()}
        else:
            dict_dores = "Sem reclamações suficientes"

        stats[empresa] = {
            "Volume Reviews": total,
            "Nota Média": round(sub['nota'].mean(), 2),
            "NPS Estimado": round(nps, 1),
            "Top 3 Problemas": dict_dores
        }

    return stats

# --- 3. RELATÓRIO ESTRATÉGICO (Soft Skills) ---
def gerar_relatorio_ia(dados):
    print("Gerando Relatório de Inteligência de Mercado...")
    model = GenerativeModel(MODEL_ID)

    prompt = f"""
    Atue como um Especialista de Mercado de Eventos.
    Compare os dois principais players: Sympla e Eventbrite (baseado nos dados abaixo).

    DADOS REAIS DO MERCADO:
    {json.dumps(dados, indent=2, ensure_ascii=False)}

    GERE UM RELATÓRIO EXECUTIVO:
    1. **O Vencedor**: Quem tem a melhor percepção de marca (NPS/Nota)?
    2. **Análise de Vulnerabilidade**:
       - Qual o "Calcanhar de Aquiles" do Sympla? (Técnico ou Taxas?)
       - Qual o "Calcanhar de Aquiles" da Eventbrite?
    3. **Insights de Produto**:
       - Se o Sympla quisesse roubar usuários da Eventbrite amanhã, o que deveria consertar primeiro?

    Use formatação Markdown. Seja direto e crítico.
    """

    response = model.generate_content(prompt)
    return response.text

# --- EXECUÇÃO ---
dados_mercado = gerar_dados_comparativos()

if dados_mercado:
    relatorio = gerar_relatorio_ia(dados_mercado)
    print("\n" + "="*50)
    print(relatorio)
    print("="*50)

📥 Baixando dados de reviews_mobile_refined_gold...
🥊 Players no Ringue: ['Sympla' 'Eventbrite']
🧠 Gerando Relatório de Inteligência de Mercado...

Prezado(a) Diretor(a),

Segue a análise comparativa executiva entre Sympla e Eventbrite, baseada nos dados de percepção de mercado fornecidos.

---

### **Relatório Executivo: Comparativo Sympla vs. Eventbrite**

Como especialista de mercado de eventos, a análise dos dados revela insights cruciais sobre a performance e a percepção de cada plataforma.

#### 1. **O Vencedor: Percepção de Marca**

Com base nos dados de percepção de marca (NPS e Nota Média), a **Eventbrite** emerge como o claro vencedor.

*   **Eventbrite:** Apresenta uma **Nota Média de 4.32** e um impressionante **NPS Estimado de 58.4**. Este NPS indica uma alta probabilidade de recomendação e lealdade por parte dos usuários.
*   **Sympla:** Possui uma **Nota Média de 3.78** e um **NPS Estimado de 27.4**. Embora aceitável, está significativamente abaixo dos patamares da Eventb

In [25]:
import vertexai
from vertexai.generative_models import GenerativeModel, GenerationConfig
from google.cloud import bigquery
import pandas as pd
import json
import warnings

# --- 1. CONFIGURAÇÃO ---
warnings.filterwarnings("ignore")
PROJECT_ID = "seu-projeto"
LOCATION = "us-central1"
DATASET_ID = "sympla"
TABLE_NAME = "reviews_mobile_refined_gold"
MODEL_ID = "gemini-2.5-flash"

vertexai.init(project=PROJECT_ID, location=LOCATION)
client = bigquery.Client(project=PROJECT_ID)

# --- 2. CÁLCULO DE PONTOS FORTES (Hard Skills) ---
def gerar_dados_comparativos_elogios():
    print(f"📥 Baixando dados de {TABLE_NAME}...")

    query = f"""
    SELECT app_source, nota, tema_principal_ai
    FROM `{PROJECT_ID}.{DATASET_ID}.{TABLE_NAME}`
    """
    df = client.query(query).to_dataframe()

    if 'app_source' not in df.columns:
        print(f"Erro: Coluna 'app_source' não encontrada.")
        return None

    # Classificação NPS
    df['nps_tipo'] = df['nota'].apply(lambda x: 'Detrator' if x <= 3 else ('Neutro' if x == 4 else 'Promotor'))

    stats = {}
    players = df['app_source'].unique()
    print(f"Players no Ringue: {players}")

    for empresa in players:
        sub = df[df['app_source'] == empresa]
        total = len(sub)

        if total == 0: continue

        # NPS e Médias
        prom = len(sub[sub['nps_tipo'] == 'Promotor'])
        detr = len(sub[sub['nps_tipo'] == 'Detrator'])
        nps = ((prom - detr) / total) * 100

        # --- LÓGICA DE ELOGIOS ---
        # Filtramos Notas Altas (4 e 5)
        # Se o tema for "Usabilidade" com nota 5, é um Ponto Forte de UX.
        # Se o tema for "Elogio", é satisfação geral.
        base_positiva = sub[sub['nota'] >= 4]

        if len(base_positiva) > 0:
            # Conta os temas mais frequentes nas avaliações positivas
            top_features = base_positiva['tema_principal_ai'].value_counts(normalize=True).head(4)
            dict_elogios = {k: f"{v*100:.1f}%" for k, v in top_features.items()}
        else:
            dict_elogios = "Sem elogios suficientes"

        stats[empresa] = {
            "Volume Reviews": total,
            "Nota Média": round(sub['nota'].mean(), 2),
            "NPS Estimado": round(nps, 1),
            "Principais Drivers de Satisfação": dict_elogios
        }

    return stats

# --- 3. RELATÓRIO DE DIFERENCIAIS (Soft Skills) ---
def gerar_relatorio_pontos_fortes(dados):
    print("Gerando Análise de Diferenciais Competitivos...")
    model = GenerativeModel(MODEL_ID)

    prompt = f"""
    Atue como um Especialista de Produto e CX.
    Compare os PONTOS FORTES do Sympla e da Eventbrite.

    DADOS DE MERCADO (Foco em Satisfação):
    {json.dumps(dados, indent=2, ensure_ascii=False)}

    GERE UM RELATÓRIO EXECUTIVO POSITIVO:
    1. **O Campeão da Experiência**: Quem tem a maior base de fãs (NPS/Nota)?
    2. **Diferencial Competitivo (Onde cada um brilha?)**:
       - Analise os "Drivers de Satisfação".
       - Se o Sympla tem muitos elogios em "Usabilidade/Busca" com nota alta, isso é uma fortaleza.
       - Se a Eventbrite tem elogios em "Acesso Ingresso", isso é confiança.
       - (Obs: O tema "Elogio" indica satisfação geral/brand love).
    3. **Conclusão de Valor**:
       - Por que os clientes AMAM a Eventbrite?
       - Por que os clientes AMAM o Sympla?

    Seja direto. Use Markdown.
    """

    response = model.generate_content(prompt)
    return response.text

# --- EXECUÇÃO ---
dados_mercado = gerar_dados_comparativos_elogios()

if dados_mercado:
    relatorio = gerar_relatorio_pontos_fortes(dados_mercado)
    print("\n" + "="*50)
    print(relatorio)
    print("="*50)

📥 Baixando dados de reviews_mobile_refined_gold...
🥊 Players no Ringue: ['Sympla' 'Eventbrite']
🧠 Gerando Análise de Diferenciais Competitivos...

## Relatório Executivo: Comparativo de Satisfação - Sympla vs. Eventbrite

Prezados,

Este relatório apresenta uma análise comparativa dos pontos fortes em satisfação do cliente para as plataformas Sympla e Eventbrite, baseando-se nos dados de mercado fornecidos. O objetivo é destacar onde cada plataforma brilha, contribuindo para uma experiência positiva do usuário.

---

### 1. O Campeão da Experiência

Ao analisarmos a base de fãs e a satisfação geral, a **Eventbrite** se destaca como a campeã da experiência:

*   **Eventbrite:** Com uma **Nota Média de 4.32** e um **NPS Estimado notável de 58.4**, a Eventbrite demonstra uma lealdade e satisfação excepcionalmente altas entre seus usuários. Embora com um volume de reviews menor (2.193), a qualidade da experiência é consistentemente superior.
*   **Sympla:** O Sympla possui uma base de usuá

In [27]:
import vertexai
from vertexai.generative_models import GenerativeModel, GenerationConfig
from google.cloud import bigquery
import pandas as pd
import json
import warnings

# --- 1. CONFIGURAÇÃO ---
warnings.filterwarnings("ignore")
PROJECT_ID = "seu-projeto"
LOCATION = "us-central1"
DATASET_ID = "sympla"
TABLE_NAME = "reviews_mobile_refined_gold"
MODEL_ID = "gemini-2.5-flash"

vertexai.init(project=PROJECT_ID, location=LOCATION)
client = bigquery.Client(project=PROJECT_ID)

# --- 2. CÁLCULO DE VERSÕES CRÍTICAS (Hard Skills) ---
def auditar_versoes_sympla():
    print(f"Baixando histórico de versões do Sympla...")

    # Filtra APENAS Sympla e apenas quem informou a versão
    query = f"""
    SELECT versao_app, nota, tema_principal_ai
    FROM `{PROJECT_ID}.{DATASET_ID}.{TABLE_NAME}`
    WHERE app_source = 'Sympla'
      AND versao_app IS NOT NULL
      AND versao_app != 'null'
    """
    df = client.query(query).to_dataframe()

    # Filtra apenas Detratores (Nota <= 3) para achar falhas
    df_crise = df[df['nota'] <= 3]

    if len(df_crise) == 0:
        print("Nenhuma reclamação com versão encontrada.")
        return None

    print(f"Analisando {len(df_crise)} reclamações com versão identificada...")

    # Top 5 Versões com mais volume de reclamação
    top_versoes = df_crise['versao_app'].value_counts().head(5)

    stats_versao = {}

    for versao, volume in top_versoes.items():
        # Pega os dados dessa versão específica
        dados_v = df_crise[df_crise['versao_app'] == versao]

        # Quais foram os 2 maiores problemas dessa versão?
        top_problemas = dados_v['tema_principal_ai'].value_counts(normalize=True).head(2)
        dict_problemas = {k: f"{v*100:.1f}%" for k, v in top_problemas.items()}

        stats_versao[str(versao)] = {
            "Volume de Queixas": int(volume),
            "O que quebrou?": dict_problemas
        }

    return stats_versao

# --- 3. RELATÓRIO DE ENGENHARIA (Soft Skills) ---
def gerar_autopsia_ia(dados):
    print("O Gemini está fazendo a autópsia das versões...")
    model = GenerativeModel(MODEL_ID)

    prompt = f"""
    Atue como Lead de Engenharia Mobile do Sympla.
    Você recebeu um relatório de bugs críticos agrupados por versão do App.

    DADOS DE FALHA (SYMPLA):
    {json.dumps(dados, indent=2, ensure_ascii=False)}

    GERE UM RELATÓRIO DE "AUTÓPSIA DE RELEASE":
    1. **A Versão Maldita**: Qual foi a versão mais instável da história recente? (Aquela com mais queixas).
    2. **Diagnóstico**: O que aconteceu nela? (Ex: "A versão X focou em problemas de Login, indicando uma falha na autenticação").
    3. **Padrão de Falha**: Existe um padrão? (Ex: "Todas as versões ruins quebram o Pagamento" ou "O problema varia a cada release?").

    Seja técnico e direto. Use Markdown.
    """

    response = model.generate_content(prompt)
    return response.text

# --- EXECUÇÃO ---
dados_sympla = auditar_versoes_sympla()

if dados_sympla:
    relatorio = gerar_autopsia_ia(dados_sympla)
    print("\n" + "="*50)
    print(relatorio)
    print("="*50)

📥 Baixando histórico de versões do Sympla...
📉 Analisando 2131 reclamações com versão identificada...
🧠 O Gemini está fazendo a autópsia das versões...

## RELATÓRIO DE AUTÓPSIA DE RELEASE - MOBILE SYMPLA

Prezados,

Segue análise técnica do relatório de bugs críticos agrupados por versão, visando diagnosticar as falhas mais impactantes e identificar padrões para ações corretivas futuras.

---

### 1. A Versão Maldita

A versão mais instável da história recente, com o maior volume de queixas, foi a **11.19.12**, registrando 59 ocorrências críticas.

### 2. Diagnóstico da 11.19.12

A versão **11.19.12** apresentou uma quebra significativa em duas áreas críticas, com impacto equitativo:
*   **Pagamento/Taxas (27.1%)**: Indica falhas diretas no fluxo de transação financeira, seja na conclusão do pagamento, aplicação de taxas, ou na validação do processo. Este é um ponto de conversão crucial e qualquer instabilidade aqui gera perda direta de receita e confiança do usuário.
*   **Usabilidad